# Abstract

# Table of Contents

# Introduction

## Background and motivation

Natural Language Processing (NLP) has come a long way with models like BERT, which help computers understand and generate human language more effectively. This progress opens up exciting possibilities for creating engaging and contextually rich stories.

The idea for this project started from a personal experience within our team. One of us had a nephew who asked for a bedtime story about "pirates in space." As Catalans who value storytelling but aren't all naturally imaginative, we struggled. Attempts to use existing generative models resulted in stories that missed the context and creativity the child wanted. This frustration highlighted a gap in current story generation tools.

Motivated by this experience, we set out to create a better way to generate fairy tales that truly capture the magic and context of the themes kids ask for. We decided to train a BERT model specifically for this purpose and use a Retrieval-Augmented Generation (RAG) system. This way, when someone requests a story, the system can understand the request, find similar tales, and generate a new, engaging story.

Our goal is to provide a tool that can make storytelling easier and more fun, ensuring that every story is as imaginative and contextually rich as the ones we cherish from our childhood.

## Objectives and scope

### Objectives

The primary objective of this project is to develop an advanced story generation system that can create contextually rich and engaging fairy tales based on user input. 

In order to achieve this, we would divide our project into 3 different phases:

![Phases of our project](initial_arquitecture.png)

These 3 phases have been further divided into more specific tasks:

**Create a Custom Tokenizer**:

- Develop a tokenizer specifically designed for our dataset of fairy tales to ensure accurate text processing.
  
**Train a BERT Model**: 

- Train a BERT model using the custom tokenizer and a diverse dataset of fairy tales to enable it to understand and generate narrative content effectively.

**Implement a RAG System**: 
- Develop a system that integrates a vector database for storing embeddings of fairy tales and uses these embeddings to enhance the story generation process.

**Generate Rich Fairy Tales**: 
- Use the embeddings retrieved from our database as context for an LLM to generate a more rich and engaging version of a fairy tale.

**Evaluate the System**: 
- Assess the performance of the system through qualitative and quantitative metrics to ensure it meets the desired objectives.

### Scope

The scope of this project encompasses the following key areas:


**Data Collection and Preprocessing:**

- Collect a dataset of fairy tales for training the BERT model.
- Preprocess the dataset to ensure it is suitable for training.

**Tokenizer Development:**

- Create a custom tokenizer tailored to the fairy tale dataset.

**Model Training and Development:**

- Train the BERT model on the collected dataset. We will do this by implementing:
  - **MLM (Masked Language Model)**
  - **NSP (Next Sentence Prediction)** tasks to fine-tune the model for narrative generation.

**Embedding and Vector Database:**

- Create embeddings of the fairy tales using the trained BERT model.
- Store these embeddings in a vector database to facilitate efficient retrieval.

**Retrieval-Augmented Generation System:**

- Develop the retrieval mechanism using cosine similarity to find relevant story contexts.
- Integrate the retrieval system with a language model to generate new stories based on user input.

**System Evaluation:**

- Conduct experiments to evaluate the retrieval accuracy and the quality of the generated stories.

**User Interface:**

- Design a user-friendly interface where users can input their story requests and receive generated stories.


# Methodology

### Data Collection

**Data Gathering**

To develop an advanced story generation system, we needed a robust and diverse dataset of fairy tales. Our data collection process involved multiple sources to ensure a comprehensive dataset. Here’s how we approached it:

1. **Kaggle**: We found several relevant datasets on Kaggle, including:
   - [Grimm’s Fairy Tales](https://www.kaggle.com/datasets/tschomacker/grimms-fairy-tales)
   - [Grimms' Brother Fairy Tale Dataset](https://www.kaggle.com/datasets/cornellius/grimms-brother-fairy-tale-dataset)

2. **Hugging Face**: We utilized datasets from Hugging Face, such as:
   - [FairytaleQA Dataset](https://huggingface.co/datasets/WorkInTheDark/FairytaleQA)
   - [FairyTales Dataset](https://huggingface.co/datasets/KyiThinNu/FairyTales)

3. **GitHub**: We accessed the FairytaleQA dataset from GitHub:
   - [FairytaleQAData](https://github.com/uci-soe/FairytaleQAData/tree/main)

4. **Web Scraping**: To supplement the datasets, we performed web scraping on various websites dedicated to fairy tales, including:
   - [Dream Little Star](https://dreamlittlestar.com/)
   - [Read the Tale](https://www.readthetale.com/)

We used web scraping tools and techniques to extract text data from these sources, carefully handling HTML parsing and cleaning the text to make it suitable for training.

By combining datasets from these sources, we compiled a rich and diverse collection of fairy tales. This dataset was then preprocessed to remove any inconsistencies and ensure it was ready for training our BERT model.

#### Dataset Analysis

To better understand the characteristics of our dataset, we performed a detailed analysis, including visualizations such as histograms. Below are some of the key insights:

- **Story Length Distribution**: The histogram below shows the distribution of story lengths in the dataset.

![Story Length Distribution](imagen3.png)

- **Vocabulary Size**: The histogram below represents the distribution of vocabulary sizes across different stories.

![Vocabulary Size Distribution](imagen2.png)

- **Average Sentence Length**: The histogram below shows the distribution of average sentence lengths in the dataset.

![Average Sentence Length](imagen1.png)

These analyses helped us to understand the dataset better and guided our preprocessing and model training steps.

#### Dataset Metrics

For training and validating our model, we have gathered a total of 1,183 fairy tales. Here are some key metrics of our dataset:

- **Total Sentences**: 128,541 sentences
- **Mean Sentences per Story**: 114 sentences
- **Total Words**: 2,631,859 words


### Tokenizer

**Byte-Pair Encoding (BPE) Tokenizer**

Initially, our team developed a Byte-Pair Encoding (BPE) tokenizer to process the fairy tales. BPE tokenization involves merging the most frequent pairs of characters or subwords iteratively until a specified vocabulary size is reached. This method helps in efficiently encoding words and subwords, which is beneficial for the training of our BERT model.

**Key steps in our BPE tokenizer implementation:**
1. **Initialization**: Define the number of merges (iterations) for the tokenization process.
2. **Vocabulary Creation**: Split words into characters and calculate the frequency of each pair of characters.
3. **Pair Merging**: Iteratively merge the most frequent pairs of characters to create subwords.

```python
for _ in range(self.num_merges):
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[(symbols[i], symbols[i + 1])] += freq
    best_pair = max(pairs, key=pairs.get)
    new_symbol = ''.join(best_pair)
    new_vocab = defaultdict(int)
    for word in vocab:
        new_word = word.replace(' '.join(best_pair), new_symbol)
        new_vocab[new_word] += vocab[word]
    vocab = new_vocab
```

**WordPiece Tokenizer**

After experimenting with the BPE tokenizer, we discovered that the WordPiece tokenizer provided better performance for our task. The WordPiece tokenizer iteratively builds the vocabulary by considering the most frequent subword pairs, similar to BPE, but with additional handling for character-level tokens.

**Key advantages of the WordPiece tokenizer:**
1. **Efficiency in handling rare words**: The WordPiece tokenizer can break down rare words into subwords more effectively, improving the model's ability to generalize.
2. **Handling of special characters**: The tokenizer includes special tokens for punctuation, spaces, and other characters, enhancing its ability to process diverse text formats.

**Key steps in our WordPiece tokenizer implementation:**
1. **Initialization**: Define the vocabulary size and special tokens.
2. **Word Frequency Calculation**: Count the frequency of words and characters in the text.
3. **Vocabulary Building**: Iteratively merge the most frequent pairs to form subwords and build the vocabulary.


```python
while len(self.vocab) < self.vocab_size:
scores = self._compute_pair_scores(splits)
best_pair = max(scores, key=scores.get)
splits = self._merge_pair(*best_pair, splits)
new_token = best_pair[0] + best_pair[1][2:] if best_pair[1].startswith("##") else best_pair[0] + best_pair[1]
self.vocab.append(new_token)
```

**Conclusion: WordPiece Tokenizer > BPE Tokenizer**

We opted for the WordPiece tokenizer over the Byte-Pair Encoding (BPE) tokenizer for several key reasons:

##### Key Differences

1. **Handling Rare Words**:
   - **BPE**: Less effective at handling rare words due to its frequency-based merges.
   - **WordPiece**: Breaks down rare words into smaller, meaningful subwords, improving generalization.

2. **Special Characters**:
   - **BPE**: Lacks explicit handling of special characters.
   - **WordPiece**: Includes special tokens for punctuation and spaces, providing accurate text representation.

3. **Vocabulary Efficiency**:
   - **BPE**: Fixed number of merges can lead to suboptimal vocabulary size.
   - **WordPiece**: Dynamically builds a balanced and efficient vocabulary.

4. **Contextual Understanding**:
   - **BPE**: May miss contextual nuances in narrative text.
   - **WordPiece**: Better contextual understanding due to granular word breakdown and handling of special characters.


The WordPiece tokenizer demonstrated superior performance in handling rare words, special characters, and providing efficient, contextually aware tokenization. These advantages make it the optimal choice for processing our fairy tale dataset and generating rich, engaging stories.


**Tokenizer Output**

Specifically, our final tokenizer outputs the following information:

- **Token IDs**: Each token in the text is mapped to a unique integer ID from the vocabulary.
- **Special Tokens**: These are tokens added to the text to provide additional information about the structure of the input.
  - **[CLS]**: Added at the beginning of the text. Represents the entire input sequence and its embedding is used for classification tasks.
  - **[SEP]**: Separates different parts of the input (e.g., question and answer, two sentences). For single sequences, it is added at the end.
  - **[PAD]**: Used to pad sequences to the same length within a batch.
- **Attention Mask**: Indicates which tokens should be attended to and which should be ignored (due to padding).
- **Token Type IDs**: Identifies different segments in the input. For single sequences, all values are typically 0. For paired sequences, the first sequence might have all 0s and the second sequence all 1s.


### Model Architecture

#### Custom BERT

##### Visual Summary

Here's the image illustrating the BERT model architecture:

![Model Architecture](bert_model.png)

Follow this section for specific explanations on every section

##### Input of our model

1. **Tokenizer Output**

**Components**:
- **Input IDs**: Unique integer IDs for each token in the text.
- **Attention Mask**: Indicates which tokens should be attended to (1) and which should be ignored (0) due to padding.
- **Segment IDs**: Identifies different segments in the input. For single sequences, all values are typically 0. For paired sequences, the first sequence might have all 0s and the second sequence all 1s.


##### Embedding Layer

The embedding layer converts input tokens into dense vectors.

```python
class EmbeddingLayer(nn.Module):
    def __init__(self, vocab_size, embed_size, seq_len=64, dropout=0.1):
        super(EmbeddingLayer, self).__init__()
        self.token_embeddings = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.segment_embeddings = nn.Embedding(3, embed_size, padding_idx=0)
        self.position_embeddings = PositionalEmbedding(d_model=embed_size, max_len=seq_len)
        self.dropout = nn.Dropout(p=dropout)
       
    def forward(self, input_ids, segment_ids):        
        x = self.token_embeddings(input_ids) + self.position_embeddings(input_ids) + self.segment_embeddings(segment_ids)
        x = self.dropout(x)
        return x
```

**Key Steps:**

- Convert token IDs to embeddings.
- Add positional embeddings to encode the position of each token. (See next section)
- Add segment embeddings to differentiate between different segments of input.
- Apply dropout for regularization.

##### Positional Encoding

3. **Positional Encoding**

Adds positional information to the embeddings to ensure the model understands the order of tokens.

```python
class PositionalEmbedding(torch.nn.Module):
    def __init__(self, d_model, max_len=128):
        super().__init__()
        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False
        for pos in range(max_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
        self.pe = pe.unsqueeze(0)
    
    def forward(self, x):
        return self.pe
```

##### Transformer Encoder Block

4. **Transformer Encoder Blocks**

Consists of multiple encoder layers that apply self-attention mechanisms and feed-forward networks.

```python
class EncoderLayer(torch.nn.Module):
    def __init__(self, d_model=768, heads=12, feed_forward_hidden=768 * 4, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.layernorm = torch.nn.LayerNorm(d_model)
        self.self_multihead = MultiHeadedAttention(heads, d_model)
        self.feed_forward = FeedForward(d_model, middle_dim=feed_forward_hidden)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, embeddings, mask):
        interacted = self.dropout(self.self_multihead(embeddings, embeddings, embeddings, mask))
        interacted = self.layernorm(interacted + embeddings)
        feed_forward_out = self.dropout(self.feed_forward(interacted))
        encoded = self.layernorm(feed_forward_out + interacted)
        return encoded
```

**Key Steps:**

 - **Self-Attention:** Allows the model to focus on different parts of the input sequence.
 - **Feed-Forward Network:** Applies a fully connected feed-forward network to each token.
 - **Layer Normalization and Dropout:** Normalizes the output and applies dropout for regularization.

##### Training Tasks

**Explanation of our training tasks:**
- For our **MLM task**, our goal is to predict some tokens that we have previously masked
- For our **NSP task**, we will try to predict wether the second sentence of our input is the following sentence of our dataset
    

###### MLM: Masked Language Model

```python
class MaskedLanguageModel(nn.Module):
    def __init__(self, hidden, vocab_size):
        super().__init__()
        self.linear = nn.Linear(hidden, vocab_size)
        self.softmax = nn.LogSoftmax(dim=-1)
    
    def forward(self, input):
        x = self.linear(input)
        x = self.softmax(x)
        return x
```

**Key Steps:**
- Use a linear followed by a softmax

###### NSP: Next sentence Predict

```python
class NextSentencePrediction(nn.Module):
    def __init__(self, hidden):
        super().__init__()
        self.linear = nn.Linear(hidden, 2)
        self.softmax = nn.LogSoftmax(dim=-1)
    
    def forward(self, input):
        x = input[:, 0]
        x = self.linear(x)
        x = self.softmax(x)
        return x
```    

**Key Steps:**
- Use only the first token ([CLS]) to predict if the next sentence follows.
- Apply a linear layer followed by a softmax for classification.

#### Transfer Learning with DistilBERT

### Training Routine

#### Train, Test Split

#### Hyperparameters

#### Scheduler

#### Dropout

#### Warmup

### Embedding Generation

#### Custom BERT

**Inference and Embedding Generation**

During inference, we use the BERT model **without** the **Next Sentence Prediction (NSP)** and **Masked Language Model (MLM)** tasks. Instead, we focus on generating meaningful embeddings for our fairy tale dataset. These embeddings capture the contextual information of the input text, which is crucial for our RAG system.

**Embedding Generation:**
- The BERT model's embedding layer and encoder layers are used to transform input text into dense vector representations.
- These embeddings encapsulate the semantic meaning and context of the input text.

**Visual Summary**
Here's the image illustrating the BERT model architecture:

![Model Architecture](bert_inf.png)

Follow this section for specific explanations on every section

```python
class BERT(nn.Module):
    def __init__(self, vocab_size, seq_len=512, d_model=768, n_layers=12, heads=12, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.n_layers = n_layers
        self.heads = heads
        self.feed_forward_hidden = d_model * 4
        self.embedding = EmbeddingLayer(vocab_size=vocab_size, embed_size=d_model, seq_len=seq_len, dropout=dropout)
        self.encoder_blocks = nn.ModuleList(
            [EncoderLayer(d_model, heads, d_model * 4, dropout) for _ in range(n_layers)])
    
    def forward(self, x, segment_info):
        mask = (x > 0).unsqueeze(1).repeat(1, x.size(1), 1).unsqueeze(1)
        x = self.embedding(x, segment_info)
        for encoder in self.encoder_blocks:
            x = encoder.forward(x, mask)
        return x
```

#### Transfer Learning with DistilBERT

### Vector DB

#### Chroma DB

#### Zilliz

### LLM

A Large Language Model (LLMs) is a computational model made of neural networks trained on a huge data corpus to perform like humans in some tasks. In our specific case, we want to focus on text generation tasks and how the model  can create good quality fairy tales. To achieve that, the LLMs use transformers inside the neural network to perform these natural language procession (NLP) tasks.

We want to find one of the best options to generate stories and text. One of the best LLMs that we can use today for storytelling is Claude 3.5 sonnet. This model is not open source, but we have an API key to perform inferences. In order to try other LLMs, we have integrated Llama 3 and GPT-3.5 turbo instruct.

Llama 3 is run locally with Ollama. We wanted to run one of them locally to see how is performing compared to the other cloud based models. Another point is to see which is the difference between a 7B parameters like the Llama 3 run locally and Claude 3.5 sonnet or GPT-3.5 Turbo instruct run in their servers and with a very huge number of parameters.

For this reason, in our project, the user has the option to perform the inference with three LLMs. Can run with Llama 3 using Ollama and Langchain, or use Claude 3.5 sonnet, or use GPT-3.5 Turbo. The first one is slower than the others because it is executed locally. Then the size matter in this case, because the output of the Llama 3 is not the same as the output of the other models. The cloud base models are larger than Llama 3 and the quality is better.

Now we will show you which is the output for the same input for each of the models:

**The input prompt:**

This is a prompt for a RAG system. I need you to create a fairy tale in Catalan following the user prompt and the context. The user prompt is: Once upon a time there was a little red riding hood who lived in a small village.

And this is the context: I once went twelve miles in a coop, and it was not pleasant travelling at all., THERE was once upon a time a couple of folks who had a son called Halvor., There was a loud yell of pain, and then the ogre dashed in front of the warrior., Once upon a time there was a fisherman who had a wife and three children., This place was said to be haunted by a cannibal goblin who took the form of an old woman., Those sounds came from a little street that led to a small village along the shore., About a mile farther on the road, there is another herd for you., Once upon a time two young men living in a small village fell in love with the same girl., There was no one in the house except an old Lapland woman who was cooking fish over an oil-lamp., Then he went on and came to a wide river over which he must go..

So the output should be in Catalan.

<br>
<br>

**This is the output with local Llama 3:**

Quan era petit, hi havia una xiqueta vermella que vivia en un petit poble. La xiqueta es deia Xisca i era filla d'un bon camperol que tenia molts animals al seu camp. Ella era molt alegre i amorosa, i sempre estava disposada a ajudar els seus veïns.

Però un dia, mentre anava a visitar la seva àvia a les muntanyes, Xisca es va trobar amb una situació molt desagradable. Va sortir del poble i va començar a caminar per una carretera llarga i estreta que duia fins al costat d'un gran llac. Hi havia un home d'una força impressionant, anomenat Torsten, que havia construït una caserna al centre del poble.

Quan Xisca es va acostar a la caserna, l'esguardava amb curiositat i inquietud. De sobte, sentí un crit agònic i llavors l'home d'una força impressionant, Torsten, va sortir de la caserna i va començar a córrer cap al poble.

La xiqueta Xisca es va trobar amb una dona vella que tenia una barba llarga i unes ulls estranys. La dona li digué: "No t'acostis, xiqueta. Aquest home és un ogre i és molt perillós". Però Xisca no la va escoltar i es va acostar a ell.

Llavors, l'ogre la va agafar i se la va emportar cap al seu castell. La xiqueta Xisca era molt espantada i no sabia què fer. Però llavors sentí una veu que li deien: "Xiqueta, no t'afegeixis. Aquest home és un ogre i és molt perillós".

La xiqueta es va despertar i es va trobar al seu propi llit a casa seva. Era tot un somni! Però Xisca sabia que l'aventura no havia acabat encara. Es preparà per anar a visitar la seva àvia a les muntanyes, ja que havia promès de fer-ho.

Però abans d'anar-se'n, va sentir una veu que li deien: "Xiqueta, no t'afegeixis. Aquest home és un ogre i és molt perillós". I llavors es va adonar que l'aventura havia començat realment!

<br>
<br>


**This is the output with GPT-3.5 Turbo instruct:**

Hi ha una vegada en un petit poble una nena amb una caputxeta vermella que vivia amb la seva àvia. Un dia, la seva àvia li va demanar que portés una cistella de menjar a la seva mare, que vivia a dotze milles de distància. La nena va acceptar, tot i que no li agradava gaire viatjar en una carrossa.

Mentre viatjava, la nena va passar per un bosc on va trobar una parella de gent gran que tenia un fill anomenat Halvor. Ells li van explicar que havien sentit un crit fort de dolor i que un ogre havia aparegut davant d'un guerrer. La nena va continuar el seu camí, pensant en aquesta història.

Finalment, va arribar a la casa de la seva mare, que era una pescadora i tenia tres fills. La nena va ajudar la seva mare a preparar el peix per al sopar i li va explicar la història de l'ogre que havia sent

<br>
<br>

**This is the output with Claude 3.5 Sonnet:**

Hi havia una vegada una Caputxeta Vermella que vivia en un petit poble vora el mar. Un dia, la seva mare li va demanar que portés un cistell de menjar a la seva àvia malalta, que vivia a l'altra banda del bosc.

Mentre caminava pel camí, la Caputxeta va sentir uns sorolls estranys que provenien d'un carreró que conduïa a un altre llogaret. Intrigada, va decidir investigar-ho.

En arribar-hi, va trobar una vella lapona que cuinava peix sobre una llàntia d'oli. La vella li va advertir sobre un goblin caníbal que rondava per aquells indrets, disfressat de dona gran.

Espantada, la Caputxeta va reprendre el seu camí, però aviat es va trobar amb un gran riu que havia de creuar. Mentre buscava la manera de passar, va veure un pescador amb la seva dona i tres fills que lluitaven contra un ogre ferotge.

La Caputxeta, valenta, va ajudar la família a espantar l'ogre, que va fugir cridant de dolor. Agraït, el pescador li va indicar un camí segur per travessar el riu i arribar a casa de la seva àvia.

Finalment, la Caputxeta va aconseguir arribar a casa de l'àvia, on va descobrir que la suposada "àvia" era en realitat el goblin caníbal disfressat. Amb enginy i l'ajuda d'un jove anomenat Halvor, que passava per allà, van aconseguir derrotar el goblin i salvar la veritable àvia.

I així, la Caputxeta va tornar al seu poble, havent viscut una gran aventura i havent fet nous amics. Des d'aquell dia, sempre explicava que viatjar dotze milles en un galliner no era gens agradable, però que les aventures que havia viscut havien valgut la pena.

<br>
<br>


As we can see, if we don't specify a length, the output of GPT-3.5 Turbo is shorter than the others. But the main difference between them is the quality of the grammar and the names that they use. From our point of view, Claude 3.5 has the best Catalan and the coherence is very good. For GPT-3.5 Turbo is also good, the Catalan, but not as good as Claude. Both, Claude and GPT-3.5 Turbo have a good output quality but are larger than Llama 3 and are executed on the cloud. Llama 3 has a poor quality of Catalan, but the coherence is good.

<br><br>

### Prompting

In the context of prompting, we want to apply different techniques to find out which is the best one for our specific RAG system. The structure of the prompt can improve significantly the LLMs output. For this reason, we want to try different techniques. In a RAG system, prompts are very important to ensure a good quality in the output. The points that we can focus on the prompt are stating the task that we want to do, providing context and specifying the desired format or language of the response.

Now we will show you different templates of prompt focusing on different points. The output for these prompt will be shown in the experiments section.

<br> <br>

**Template 1: Detailed Instruction with Format Emphasis**

You are a creative storyteller. Using the following user prompt and context, craft a fairy tale in Catalan. Ensure the story is engaging and follows a clear structure with a beginning, middle, and end. User prompt: {userPrompt}. Context: {context}. Please write the story in Catalan.

<br> <br>

**Template 2: Emphasizing Creativity and Contextual Relevance**

As a master storyteller, your task is to weave an enchanting fairy tale in Catalan. Use the provided user prompt and context to inspire your story. The user prompt is: {userPrompt}. Here is the context for reference: {context}. Let your imagination run wild, and remember to write the tale in Catalan.

<br> <br>

**Template 3: Focus on Emotional Engagement and Language**

Imagine you are telling a magical fairy tale in Catalan, inspired by the user prompt and context given. The user prompt is: {userPrompt}. The context to guide your story is: {context}. Create a captivating narrative that evokes wonder and delight, and ensure it's written in Catalan.

<br> <br>

**Template 4: Concise and Direct with Contextual Clarity**

Create a fairy tale in Catalan based on the following details. User prompt: {userPrompt}. Context: {context}. Ensure the story is imaginative and written in Catalan.

<br> <br>

**Template 5: Encouraging Detailed and Vivid Storytelling**

You are an expert in crafting vivid fairy tales. Using the user prompt and context provided, write a detailed and imaginative fairy tale in Catalan. User prompt: {userPrompt}. Context: {context}. Make sure the story is full of rich descriptions and engaging elements, and is written in Catalan.

<br> <br>



### UI

All RAG systems should have a UI to be used by the users and fulfil their use cases. To achieve this, we have created a UI with Streamlit that can handle two flows.
First flow is to fulfil the user use case. Where the user types a prompt and selects the preferred LLM and the system generates a fairy tale. The other flow has the same information but shows the prompt tokenization, which context the system retrieve, which is the system prompt and the final output of the LLM.

![Initial state UI](initial_state_ui.png)


![UI first flow](UI_first_flow.png)


![UI second flow](UI_second_flow.png)


# Experiments & Results

### Tensorboard for metric follow-up

### Challenges that we faced

#### Introduction

#### Token Length adaptation

In our first iterations of training, we would have the issue where the combined loss of our training tasks would return NaN at what it seemed to be completely random iterations. 

Sometimes, it would be in our second epoch. Sometimes later. This gave us a hard time spotting the specific place where this error was coming from.

**Solution**

At that point, the maximum length of our tokens was 512. Most of the sentences that we were processing had a lower token count than that. So, in most of those cases, we would then add [PAD] (padding) tokens to make up for the difference up to 512 tokens. 

However, we eventually realized, through extensive search of the specific embeddings that were crashing, that whenever the combination of embedding of the 2 sentences was large enough (exactly 512), no [PAD] would be added. In those cases, our attention Mask would fail to understand which sections of the sentence required attention and which not. 

We solved it by making sure that, whenever a sentence was combined with another sentece that had exactly 512 tokens, we would then just look for another sentece that did not cause this issue. Combinations that were longer than 512 were discarded from the start.

#### Labeling of Masked Tokens

After debugging these issues, our training finally started without errors. However, we started to relize that our model was failing to learn properly:

**Tokenizer modification**

On of our initial approaches was to switch our Tokenizer for the original BERT tokenizer, to make sure that our custom tokenizer wasnt the problem. However, after making the change, we realized that the training process would still stagnate eventually

<img src="loss_0.png" alt="Loss of our first iterations of training" style="width:50%;">

<img src="Acc_0.png" alt="Accuracy of our first iterations of training" style="width:50%;">

- **Blue & Grey** lines are the loss using the BERT tokenizer
- **Green** line used our custom tokenizer

**Some hyperparameter modifications**

We then tried to increase the size of the model as well as adding some quality of life modifications for our model.

*We applied the following modifications:
- Batch Size increase
- Modification of segment IDs 
- More epochs
- Adding Warmup & Scheduler
 

<img src="loss_1.png" alt="Loss" style="width:50%;">

<img src="Acc_1.png" alt="Accuracy" style="width:50%;">

Even though it looked much better than before, our Loss continued to stagnate, and the accuracy of our MLM task was very low, clearly indicating that the model was not yet learning.

**Solution**

we finally realized that there was a problem in the masking inside of the encoder

**Padding Mask:** In the encoder, there was a mask used for padding that was incorrectly set up. The padding mask should typically ignore the padding tokens (usually set to **False** for padding and **True** for actual content), but it was the other way around (padding was marked as **True** and content as **False**).

We then, set up an experiment with batch size 1, in order to see if the model would overfit. And it did:


<img src="Acc_2.png" alt="Accuracy" style="width:30%;">

**New Metric**
- At this point, we also decided to add a top 5 token accuracy

After performing another complete training, we saw that our training was already not stagnating as much

<img src="Acc_3.png" alt="Accuracy" style="width:30%;">
<img src="Acc_4.png" alt="Accuracy" style="width:30%;">
<img src="loss_2.png" alt="Accuracy" style="width:30%;">

#### NSP Continues to be an issue

At this point, our MLM task was learning preety well, and it was all about tuning the hyperparamenters.
However we observed that our NSP (Next sentence predict) task was still completely random.

- This indicated that even though our model was learning the vocabulary of our tokenizer, it would not understand the semantic meaning of the sentences good enough to be able to tell if a sentence was close to another sentence.

**Weighted Loss**

One of the first things we tried was to give different weigths to the losses of the NSP task and the MLM task in order to force the model to have a better performance in the NSP task.

This did not really work. 

**Learing Rate and other hyperparameters**

 - We tried lowering the LR even though our scheduler would do the same thing
 - Increased hidden size of our forward

**Dataset Issues**

One of our fears from the start was that our dataset would not be good enough for this complex task.
At the end of the day, we finally realized that this was the case.

Roughly 1.200 fairy tales is not enough for a model to learn the ropes of a language from scratch.

At this point we were very disappointed, but we had to improvise a solution to make the model work. 

**Transfer Learning**



### Description of the experiments performed

### Results of the model

# Conclusions & further exploration

In [3]:
## Distil BERT vs Custom BERT